# <center> Тестовое задание по графовым базам данных

Для выполнения задания были установлены **Neo4j Browser** и **Nebula Graph**.

Демонстрация установленого **Nebula Graph**:
![nebula](nebula-console.png)

Демонстрация установленного **Neo4j Browser**:
![neo4j](neo4j.png)

В дальнейшем для выполнения задания используется **Neo4j Browser**.

## <center> Заполнение базы данных

Создадим ограничения на уникальность для событий и людей:

``` cypher
CREATE CONSTRAINT Event_id IF NOT EXISTS
FOR (e:Event)
REQUIRE e.eventId IS UNIQUE;
     
CREATE CONSTRAINT Person_fullName IF NOT EXISTS
FOR (p:Person)
REQUIRE p.fullName IS UNIQUE;
```

Внесем данные из источника в графовую базу данных:

``` cypher
CALL {
    LOAD CSV WITH HEADERS
    FROM 'https://disk.yandex.ru/d/s6wWqd8Ol_5IvQ' AS row 
    FIELDTERMINATOR ';'
    WITH row
    MERGE (e:Event {eventId: toInteger(row.`id события`)})
    MERGE (p1:Person {fullName: row.`ФИО участника события 1`})
    MERGE (p2:Person {fullName: row.`ФИО участника события 2`})
    MERGE (p1)-[:PARTICIPATED_IN]->(e)
    MERGE (p2)-[:PARTICIPATED_IN]->(e)
}
```

Посмотрим на получившееся графовое представление (по умолчанию Neo4j показывает небольше 300 узлов и не больше 100 соседей для каждого)
``` cypher
MATCH (n) RETURN n
```

[<img src="graph0.png" width="400"/>](graph0.png)

## <center> Запросы к базе данных

Найдем человека, участвовавшего в максимальном количестве событий:

``` cypher
MATCH (p:Person)-->(e:Event)
RETURN p, COLLECT(e) as events
ORDER BY SIZE(events) DESC LIMIT 1
```

[<img src="graph1.png" width="400"/>](graph1.png)

Этот человек - Ахромеева Алина Ивановна. Найдем всех встреченных ею людей:

``` cypher
MATCH (p1:Person {fullName: 'Ахромеева Алина Ивановна'})-->(e:Event)<--(p2:Person)
WITH COLLECT(p2) as acquaintances
UNWIND acquaintances AS acquintance
RETURN acquintance.fullName
```    

Посмотрим на первых 5 человек:

In [1]:
import pandas as pd

df = pd.read_csv("export1.csv")
df.head(5)

,acquintance.fullName
0,Андриевская Марина Ринатовна
1,Камилов Дамир Павлович
2,Арбачаков Филипп Андреевич
3,Тяжлов Ринат Владиславович
4,Дудыкина Мария Романовна


Найдем события, в которых участвовало больше двух человек:

``` cypher
MATCH (p:Person)-->(e:Event)
WITH e, COLLECT(p) as participants
WHERE SIZE(participants) > 2
RETURN e, participants
```

[<img src="graph2.png" width="400"/>](graph2.png)

## <center> Использование алгоритмов

### <center> Degree Centrality

Создадим проекцию:

``` cypher
CALL gds.graph.project('proj', ['Person', 'Event'], 'PARTICIPATED_IN')
```

Найдем 5 человек, участвовавших в максимальном количестве событий:

``` cypher
CALL gds.degree.stream('proj')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).fullName AS person, score AS numberOfEvents
ORDER BY numberOfEvents DESC, person LIMIT 5
```

In [2]:
import pandas as pd

df = pd.read_csv("export2.csv")
df

,person,numberOfEvents
0,Ахромеева Алина Ивановна,50
1,Башнина Антонина Глебовна,14
2,Медведева Дарья Алексеевна,6
3,Диомидов Игорь Ильдарович,5
4,Зимнухова Карина Даниловна,5


### <center> Louvain Community Detection

Создадим проекцию с ненаправленными отношениями:

``` cypher
CALL gds.graph.project(
    'path-proj', 
    ['Person', 'Event'], 
    {PARTICIPATED_IN: {orientation: 'UNDIRECTED'}}
)
```

С помощью алгоритма Louvain Community Detection разделим узлы на группы:

``` cypher
CALL gds.louvain.mutate('path-proj', {mutateProperty: 'communityId'})
```

Определим идентификаторы пяти самых больших групп:

``` cypher
CALL gds.graph.nodeProperty.stream('path-proj', 'communityId', ['Person'])
YIELD nodeId, propertyValue
WITH propertyValue AS communityId, size(collect(nodeId)) AS nodesCount
RETURN communityId, nodesCount ORDER BY nodesCount DESC LIMIT 5
```

In [3]:
import pandas as pd

df = pd.read_csv("export3.csv")
df

,communityId,nodesCount
0,7979,53
1,10959,15
2,5834,13
3,1098,7
4,9511,7


Посмотрим на две самые большие группы:

``` cypher
CALL gds.graph.nodeProperty.stream('path-proj', 'communityId', ['Person', 'Event'])
YIELD nodeId, propertyValue
WITH gds.util.asNode(nodeId) AS n, propertyValue AS communityId
WHERE communityId  IN [7979, 10959]
RETURN n
```

[<img src="graph3.png" width="500"/>](graph3.png)

Эти группы формируются вокруг людей, участвовавших в большом количестве событий.